In [678]:
import pymssql
import random
from faker import Faker
fake = Faker()
size=50
import datetime

In [679]:
conn = pymssql.connect("mssql.iisg.agh.edu.pl", "mstanisz", "password-removed", "mstanisz_a")
cursor = conn.cursor()

In [680]:
def clearDatabase():
    cursor.callproc('ClearDB')
clearDatabase()


In [681]:
def sqlyDate(date):
    return date.strftime('%Y-%m-%d')

    @FirstName varchar(255),
	@LastName varchar(255),
	@BirthDate date,
	@phone varchar(10),
	@email varchar(255),
	@address varchar(255)
    
    @ContactName varchar(255),
	@phone varchar(10),
	@email varchar(255),
	@address varchar(255),
	@nip varchar(10)
    
# Adds clients

In [682]:
privateclients=[]
companyclients=[]
for i in range(size):
    if(random.random()>0.2):
        cursor.callproc('AddPrivateClient',(fake.first_name(),fake.last_name(),sqlyDate(fake.date_time_between(start_date="-80y", end_date="-20y", tzinfo=None)),fake.phone_number(),fake.email(),fake.address()))
        privateclients.append(i+1)
    else:
        cursor.callproc('AddCompanyClient',(fake.company(),fake.phone_number(),fake.email(),fake.address(),random.randrange(1000000000,9999999999)))
        companyclients.append(i+1)

	@name varchar(255),
	@startDate date,
	@length int,

	@capacity int,
	@basePrice money,
	@studentDiscount float(10)

# Adds conferences

In [683]:
conferences=[]
conferencesdata=[]
for i in range(size/10):
    data=("Conference on "+fake.uri_page()+" by "+fake.name(),fake.date_time_between(start_date="now", end_date="+3y"),random.randrange(2,8),random.randrange(300,1000),random.randrange(50,200),random.random()/2)
    cursor.callproc('AddConference',data)
    conferencesdata.append(data)
    conferences.append(i+1)
    

    @clientID int,
	@conferenceID int,
	@participantsCount int
Add details:

    @ClientID int,
	@ConferenceID int,
	@participantNumber int,
	@firstName varchar(255),
	@lastName varchar(255),
	@phone varchar(255)

# Participants and Days

In [684]:
#Adds participants
currid=1
participants=[]
for i in privateclients:
    for j in conferences:
        cursor.callproc('AddParticipants',(i,j,1))
        participants.append((currid,i,j,1))
        if(random.random()>0.1):
                cursor.callproc('AddDetails',(i,j,1,fake.first_name(),fake.last_name(),fake.phone_number()))
        if(random.random()>0.8):
                cursor.callproc('AddStudentIdNumber',(i,j,1,random.randrange(1000000,9999999)))
        currid+=1

for i in companyclients:
    for j in conferences:
        count=random.randrange(1,8)
        cursor.callproc('AddParticipants',(i,j,count))
        for k in range(count):
            participants.append((currid+k,i,j,k+1))
            if(random.random()>0.3):
                cursor.callproc('AddDetails',(i,j,k+1,fake.first_name(),fake.last_name(),fake.phone_number()))
            if(random.random()>0.9):
                cursor.callproc('AddStudentIdNumber',(i,j,k+1,random.randrange(1000000,9999999)))
        currid+=count

In [685]:

for i in participants:
    for day in range(conferencesdata[i[2]-1][2]):
        if(random.random()>0.7):
            cursor.callproc('AddDayForParticipant',(i[1],i[2],i[3],day+1))

# Workshop definitions and workshops

In [686]:
#add definitons
workshopsdefinitons=[]
for i in range(size/5):
    data=("Workshop on "+fake.uri_page()+" by "+fake.name(),random.choice(['01:30:00','02:00:00','01:00:00']),random.randrange(10,100))
    cursor.callproc('AddWorkshopDefinition',data)
    workshopsdefinitons.append(i+1)

#Adds workshops
workshops=[]

for i in range(size/2):
    conferenceid = random.choice(conferences)
    conference= conferencesdata[conferenceid-1]
    day=random.randrange(0,conference[2]-1)
    dtm=datetime.datetime.combine(conference[1] + datetime.timedelta(days=day), 
                          datetime.time(random.randrange(8,18), 0))
    data=(conferenceid,random.choice(workshopsdefinitons),dtm,None,None,fake.address(),random.randrange(200,300))
    cursor.callproc('AddWorkshop',data)
    workshops.append((i+1,conferenceid,dtm,day))

# Workshops for participants

In [689]:
ordered=[]
for workshop in workshops:
    cursor.execute("SELECT participantID FROM ParticipantDayChoices WHERE conferenceDayID = (select conferenceDayID from ConferenceDay where conferenceID = " + str(workshop[1]) + "and day = "+str(workshop[3])+")")
    for i in cursor.fetchall():  
        data=(i[0],None,None,None,workshop[0])
        if((workshop[3],i[0]) not in ordered):
            ordered.append((workshop[3],i[0]))
            cursor.callproc('AddWorkshopForParticipant',data)

# Pricing steps

In [690]:
for conferenceid in conferences:
    data=conferencesdata[conferenceid-1]
    initialprice=1.0
    daysbefore=0
    daysbeforeend=random.randrange(20,40)
    while(random.random()>0.4):
        initialprice=initialprice*(random.random()/2+0.5)
        data=(conferenceid,daysbeforeend,daysbefore,initialprice)
        cursor.callproc('AddPricingStep',data)
        daysbefore=daysbeforeend
        daysbeforeend+=random.randrange(20,40)

In [691]:
conn.commit()
conn.close()